# Tecnológico de Monterrey
## Maestría en Inteligencia Artificial Aplicada
---

### **Materia:** Proyecto Integrador

---

### **Avance 2:** Ingeniería de características

---

### **Proyecto:** Chatbot jurídico para soporte informativo en el análisis de carpetas de investigación

---

### **Profesora:** Dra. Grettel Barceló Alonso
### **Asesor:** Dr. Horacio Martínez Alfaro

---
### **Equipo 42 - Autorización individual**
### A00841954 Christian Erick Mercado Flores

---

Fecha: 05 de Octubre de 2025

### **Descripción del Proyecto**

En el marco del desarrollo de la materia de Proyecto Integrador, el presente trabajo propone el desarrollo del proyecto que lleva por título: **Chatbot jurídico para soporte informativo en el análisis de carpetas de investigación.**

La institución con la cual se colaborará y donde se implementará el proyecto es la **Fiscalía Especializada en Combate a la Corrupción del Estado de Michoacán**, también conocida como la **Fiscalía Anticorrupción de Michoacán**.

Específicamente, el área dentro de la institución donde se pondrá en marcha el trabajo es la **Unidad de Delitos Cometidos por Servidores Públicos y Particulares**.

### **Objetivo del Proyecto**

Poner a disposición de los agentes de la **Unidad de Delitos Cometidos por Servidores Públicos y Particulares** de la **Fiscalía Anticorrupción de Michoacán**, un **chatbot conversacional** que pueda proveer **soporte informativo** a partir del **marco jurídico aplicable** y de **carpetas de investigación** centralizados en un sistema de archivos.

### **Descripción del trabajo**

El presente trabajo realiza un proceso de ingeniería de características sobre los datos textuales ya tratados y analizados previamente por medio de un estudio *EDA*. Así, dentro de este mismo trabajo en una primera etapa se cargan los documentos del marco jurídico y de las carpetas de investigación. Después, se extrae el texto de los documentos y se aplica un proceso de *tokenización* y *chunking* con la meta de limpiar ese texto.

Una vez completada la anterior limpieza descrita, el trabajo realiza un proceso de *Feature Engineering* que consta de la justificación de la elección de un modelo de *Deep Learning* que funge como modelo de vectorización o *Embedding Model*. Este modelo transforma el texto tratado previamente para ser vectorizado en arreglos de números conocidos como *embeddings*. Estos últimos, representan las características que suministrarán a un sistema *RAG* de elementos para construir un espacio vectorial, sobre el cual realizar un proceso de recuperación basado en cálculos de similitud coseno o *cosine similarity* para luego suministrar a un *LLM* de contexto.

# Índice

#### 1. Librarías y Constantes

#### 2. Construcción de Corpus de Texto

#### 3. Preprocesamiento de Corpus de Texto

#### 4. Ingeniería de Características (*Feature Engineering*)
	* 4.1. Normalización
	* 4.2. Selección

#### 5. Conclusiones

#### 6. Referencias

# 1. Librarías y Constantes

In [1]:
pip install -q pandas numpy matplotlib seaborn tqdm tika pdfplumber python-docx pillow spacy python-magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 141.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 118.3 MB/s eta 0:00:00


In [2]:
# Importación de Librearías

# 1) Librerías Estándar de Python
import math
import mimetypes
import os
import re
import unicodedata
from datetime import datetime
from pathlib import Path

# 2) Librerías de Terceros
import magic
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pdfplumber
import seaborn as sns
import torch
from docx import Document
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import stats
from scipy.stats import kurtosis, skew
from tika import parser as tika_parser
from torch import Tensor
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

# 3) Configuración y Descargas
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# Configuración de Constantes

# 1) Definición de la carpeta base
BASE_DIR = Path("./file-system")

# 2) Definición de subcarpetas específicas
CARPETAS_DIR = BASE_DIR / "CARPETAS"
MARCO_LEGAL_DIR = BASE_DIR / "MARCO-LEGAL"

# 3) Configuración del conjunto de stopwords en español
SPANISH_STOP = set(stopwords.words('spanish')) if 'spanish' in stopwords.fileids() else set()

# 2. Construcción de Corpus de Texto

En esta sección se procedió a la **construcción de un corpus de textos** que representa los datos con lo cuales el proyecto "Chatbot jurídico para soporte informativo en el análisis de Carpetas de Investigación" va a trabajar. Este corpus está integrado por los siguientes tipos de documentos:
  - Documentos del **Marco Jurídico**: contienen textos que están divididos estructuralmente siguiendo una lógica jurídica (como capítulos, secciones, artículos, etc.). Provienen de instituciones, entidades y fuentes legales (como Código Penal para el Estado de Michoacán, Ley Orgánica de la Fiscalía General del Estado de Michoacán, etc.).

  - Documentos de **Carpetas de Investigación**: tienen textos escritos siguiendo una narrativa legal o jurídica, o siguiendo formatos administrativo-legales. Se generan dentro de la propia Unidad de la Fiscalía con la que se trabaja en el proyecto.

In [4]:
def process_files(base_path):
    """
    Recorre recursivamente los archivos dentro de un directorio base y construye
    un DataFrame con metadatos de cada archivo encontrado.

    Parámetros:
        base_path (Path) : Objeto Path que indica el directorio base a recorrer.

    Retorno:
        pd.DataFrame : DataFrame que contiene una fila por archivo con las siguientes
                       columnas:
                       - "path"       (str)  : Ruta completa del archivo.
                       - "name"       (str)  : Nombre base del archivo sin extensión.
                       - "size_bytes" (int)  : Tamaño del archivo en bytes.
                       - "mime"       (str)  : Tipo MIME detectado o estimado del archivo.
    """

    # 1) Inicialización de estructura para almacenar resultados
    rows = []  # Lista donde se guardarán los metadatos de cada archivo

    # 2) Recorrido recursivo de todos los elementos en el directorio base
    for p in base_path.rglob("*"):  # Itera sobre todos los archivos y carpetas
        if p.is_file():             # Solo procesa si es archivo

            # 3) Obtención de metadatos básicos del archivo
            stat = p.stat() # Se obtiene información del sistema sobre el archivo
            mime = None     # Inicializa el valor MIME como None

            # 4) Intento de detección precisa del tipo MIME
            try:
                mime = magic.from_file(str(p), mime=True)  # Detecta MIME con librería 'magic'
            except Exception:
                mime = mimetypes.guess_type(p)[0]  # Estima MIME como fallback usando 'mimetypes'

            # 5) Almacenamiento de resultados en la lista de filas
            rows.append({
                "path": str(p),             # Ruta completa del archivo
                "name": p.stem,             # Nombre del archivo sin extensión
                "size_bytes": stat.st_size, # Tamaño en bytes
                "mime": mime,               # Tipo MIME detectado o estimado
            })

    # 6) Conversión de resultados a DataFrame
    return pd.DataFrame(rows)  # Retorna la estructura tabular con metadatos

In [5]:
# Procesamiento de archivos y muestra de estadísticas

# 1) Obtención de DataFrames con los archivos procesados
carpetas_df = process_files(CARPETAS_DIR)
marco_juridico_df = process_files(MARCO_LEGAL_DIR)

# 2) Impresión de resultados de conteo de archivos procesados
print(f"=== Archivos Procesados ==")
print(f"Archivos de Carpetas de Investigación: {len(carpetas_df)}")
print(f"Archivos de Marco Jurídico: {len(marco_juridico_df)}")

=== Archivos Procesados ==
Archivos de Carpetas de Investigación: 53
Archivos de Marco Jurídico: 24


In [6]:
# Vista previa de los primeros registros
carpetas_df.head()

,path,name,size_bytes,mime
0,file-system/CARPETAS/2025-MICH-FECC-0001789/4-...,oficio_a_la_contraloria_del_estado,78134,application/pdf
1,file-system/CARPETAS/2025-MICH-FECC-0001789/4-...,correos_electronicos_asegurados,131893,application/pdf
2,file-system/CARPETAS/2025-MICH-FECC-0001789/4-...,oficio_a_la_secretaria_de_finanzas_y_administr...,100998,application/pdf
3,file-system/CARPETAS/2025-MICH-FECC-0001789/4-...,oficio_a_la_comision_nacional_bancaria_y_de_va...,110666,application/pdf
4,file-system/CARPETAS/2025-MICH-FECC-0001789/4-...,acta_de_transcripcion_de_audio,161698,application/pdf


In [7]:
# Información general del DataFrame
carpetas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   path        53 non-null     object
 1   name        53 non-null     object
 2   size_bytes  53 non-null     int64 
 3   mime        53 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.8+ KB


In [8]:
# Vista previa de los primeros registros
marco_juridico_df.head()

,path,name,size_bytes,mime
0,file-system/MARCO-LEGAL/TIPO-DELITO/ley_organi...,ley_organica_de_la_fiscalia_general_del_estado...,616234,application/pdf
1,file-system/MARCO-LEGAL/TIPO-DELITO/codigo_pen...,codigo_penal_para_el_estado_de_michoacan,1940655,application/pdf
2,file-system/MARCO-LEGAL/TIPO-DELITO/reglamento...,reglamento_de_la_ley_organica_de_la_fiscalia_g...,1382808,application/pdf
3,file-system/MARCO-LEGAL/PROCESO-PENAL/convenci...,convencion_americana_sobre_derechos_humanos,157276,application/pdf
4,file-system/MARCO-LEGAL/PROCESO-PENAL/corte_in...,corte_interamericana_de_derechos_humanos_caso_...,73003,application/pdf


In [9]:
# Información general del DataFrame
marco_juridico_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   path        24 non-null     object
 1   name        24 non-null     object
 2   size_bytes  24 non-null     int64 
 3   mime        24 non-null     object
dtypes: int64(1), object(3)
memory usage: 900.0+ bytes


# 3. Preprocesamiento de Corpus de Texto

En esta sección se procedió con el **preprocesamiento del corpus de texto**. Este proceso es sustancialmente diferente al preprocesamiento de los tipos de datos numéricos o categóricos. Pues, el procesar los textos no se trata de una tarea de buscar valores atípicos, faltantes o imputar cifras, sino que la tarea en realidad es preservar tanto la estructura lógica e integridad semántica de los textos (Ferraris et al., 2024), toda vez que son procesados para que los modelos de *Machine Learning* puedan utilizarlos. Para esta tarea de preprocesamiento se utilizaron técnicas de Procesamiento de Lenguaje Natural o *Natural Language Processing* (*NLP*). Específicamente se aplicaron técnicas de *Tokenización* (dividir el texto en unidades más pequeñas como palabras, subpalabras o caracteres) y *Chunking* (dividir fragmentos de texto o *tokens* en segmentos) (Vaj, 2024).

Los procesos que se implementaron fueron los siguientes:

  - ***Tokenización***: tanto los documentos del **Marco Jurídico** como los de las **Carpetas de Investigación** representan textos que siguen una lógica narrativa y descriptiva propia del campo legal. Aunque los documentos del **Marco Jurídico** presentan estructuras más rígidas (como artículos) y los del las **Carpetas de Investigación** tiene una narrativa más administrativa y legal, en ambos casos, la puntuación, los saltos de línea y la estructura son importantes para mantener el contexto semántico de los textos. En este sentido, se procedió con una limpieza de texto más simple para que el contenido no se perdiera, aplicando normalizando Unicode, convirtiendo a minúsculas y preservando puntuación.

  - ***Chunking***: para cada tipo de documento se siguió una division de *chunks* distinta, derivado de la naturaleza de los mismos. Sin embargo, al realizar el *overlapping* entre *tokens* de los *chunks* para preservar el contexto, se siguió la misma técnica que fue basada en el método descrito en el artículo "*No Argument Left Behind: Overlapping Chunks for Faster Processing of Arbitrarily Long Legal Texts*" (Fama et al., 2024). El proceso de *overlapping* aplicado fue calcular un porcentaje de *tokens* al inicio y final de cada *chunk*, representada por la métrica `Z`, para hacer un *overlapping* con los vecinos adyacentes, creando así una superposición simétrica de `Z/2` *tokens* en los límites de cada *chunk* adyacente para preservar el contexto. Las dimensiones de los *chunks* fueron determinadas de la siguiente manera para cada documento:

    * Documentos de **Carpetas de Investigación**: estos documentos no presentan una estructura larga de *tokens* ni presenta estructuras rígidas como artículos. Por lo tanto, se determinó una dimension estándar de 512 posiciones de *tokens* por *chunk* con el objetivo de granular el contexto, pero sin dividirlo mucho, aplicando un *overlapping* de 5%.

    * Documentos del **Marco Jurídico**: estos documentos presentan estructuras más rígidas, por lo que se procedió a generar *chunks* basados en estructuras mediante la identificación de estas a través de expresiones regulares. Debido a que estas mismas estructuras son muy variables en tamaño, se determinó generar límites de dimensión para las posiciones de *tokens* en los *chunks*. Así, se estipuló una dimensión mínima de 512 posiciones estándar y un máximo de 4,096 *tokens* que representan de las ventanas más grandes propuestas en la literatura, aplicando un *overlapping* de 5%.


In [10]:
def extract_text_pdf(path):
    """
    Extrae el contenido textual de un archivo PDF.

    Parámetros:
        path (str) : Ruta al archivo PDF a procesar.

    Retorno:
        str : Texto completo extraído del PDF. Si ocurre un error, retorna cadena vacía.
    """

    # 1) Manejo de errores generales
    try:

        # 2) Inicialización de lista para almacenar texto de cada página
        texts = []

        # 3) Apertura del PDF y extracción de texto por página
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                texts.append(page.extract_text() or "") # Extrae texto o agrega cadena vacía

        # 4) Unión de todos los textos en un solo string
        return "\n".join(texts)

    # 5) Retorno en caso de error durante la extracción
    except Exception as e:
        return ""

In [11]:
def extract_text_docx(path):
    """
    Extrae el contenido textual de un archivo DOCX.

    Parámetros:
        path (str) : Ruta al archivo DOCX a procesar.

    Retorno:
        str : Texto completo extraído del DOCX. Si ocurre un error, retorna cadena vacía.
    """

    # 1) Manejo de errores generales
    try:

        # 2) Apertura del documento DOCX
        doc = Document(path)

        # 3) Extracción del texto de cada párrafo
        parts = [p.text for p in doc.paragraphs]

        # 4) Unión de todos los textos en un solo string
        return "\n".join(parts)

    # 5) Retorno en caso de error durante la extracción
    except Exception:
        return ""

In [12]:
def extract_text_tika(path):
    """
    Extrae el contenido textual de un archivo usando Apache Tika.

    Parámetros:
        path : Ruta al archivo a procesar con Tika.

    Retorno:
        str : Texto completo extraído del archivo. Si ocurre un error, retorna cadena vacía.
    """

    # 1) Manejo de errores generales
    try:

        # 2) Parseo del archivo usando Tika
        parsed = tika_parser.from_file(path)
        content = parsed.get('content') or parsed.get('text') or "" # Obtiene contenido principal

        # 3) Decodificación si el contenido está en bytes
        if isinstance(content, bytes):
            try:
                content = content.decode('utf-8', errors='ignore')
            except Exception:
                content = str(content)  # Fallback a string estándar si falla decodificación

        # 4) Retorno del contenido extraído
        return content or ""

    # 5) Retorno en caso de error durante la extracción
    except Exception:
        return ""

In [13]:
def extract_text(path):
    """
    Extrae el texto de un archivo aplicando primero métodos específicos por tipo
    de archivo y, si fallan, utiliza Tika como fallback.

    Parámetros:
        path (str) : Ruta al archivo a procesar.

    Retorno:
        str : Texto completo extraído del archivo, limpio de espacios iniciales y finales.
    """

    # 1) Determinar la extensión del archivo
    ext = Path(path).suffix.lower()
    text = ""

    # 2) Procesamiento de PDFs
    if ext == '.pdf':
        text = extract_text_pdf(path)       # Intento preferido con pdfplumber
        if not text:
            text = extract_text_tika(path)  # Fallback a Tika si falla

    # 3) Procesamiento de archivos Word
    elif ext in ['.docx', '.doc']:
        text = extract_text_docx(path)      # Intento preferido con python-docx
        if not text:
            text = extract_text_tika(path)  # Fallback a Tika si falla

    # 4) Procesamiento de archivos de texto plano o Markdown
    elif ext in ['.txt', '.md']:
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()             # Lectura directa de archivo
        except Exception:
            text = extract_text_tika(path)  # Fallback a Tika si falla

    # 5) Otros tipos de archivo
    else:
        text = extract_text_tika(path)      # Uso directo de Tika

    # 6) Retorno del texto limpio
    return (text or "").strip()

In [14]:
def generate_tokens(text):
    """
    Tokeniza un texto realizando normalización Unicode, manejo de saltos de línea
    y filtrado de tokens vacíos. Convierte saltos de línea en tokens especiales
    para preservar la estructura del texto.

    Parámetros:
        text (str) : Texto de entrada a tokenizar.

    Retorno:
        list : Lista de tokens procesados, incluyendo '\n' para saltos de línea.
    """

    # 1) Verificar texto vacío
    if not text:
        return []

    # 2) Normalización Unicode y conversión a minúsculas
    text = unicodedata.normalize('NFKC', text).lower()

    # 3) Reemplazar saltos de línea por un marcador temporal y limpiar espacios múltiples
    text = text.replace('\n', ' NEWLINE_TOKEN ')
    text = re.sub(r'\s+', ' ', text).strip()

    # 4) Tokenización básica usando word_tokenize
    tokens = word_tokenize(text)

    # 5) Reemplazar marcador por '\n' y filtrar tokens vacíos
    final_tokens = []
    for t in tokens:
        if t.lower() == 'newline_token':  # Reemplaza el marcador por salto de línea
            final_tokens.append('\n')
        elif t.strip():                   # Evitar tokens vacíos
            final_tokens.append(t)

    # 6) Retorno de la lista final de tokens
    return final_tokens

In [15]:
def generate_chunks(tokens, chunk_size=512, overlap_percent=0.05):
    """
    Genera chunks con solapamiento siguiendo el método del artículo
    'No Argument Left Behind: Overlapping Chunks for Faster Processing of Arbitrarily Long Legal Texts'.

    Parámetros:
        tokens (list)           : Lista de tokens.
        chunk_size (int)        : Tamaño máximo de tokens por chunk.
        overlap_percent (float) : Porcentaje de solapamiento entre chunks (0 a 1).

    Retorno:
        list : Lista de chunks con solapamiento.
    """

    # 1) Verificar que la lista de tokens no esté vacía
    if not tokens:
        return []

    # 2) Inicialización de lista de chunks y cálculo de parámetros
    chunks = []                                     # Lista donde se almacenarán los chunks generados
    total_tokens = len(tokens)                      # Número total de tokens
    z = int(chunk_size * overlap_percent)           # Número de tokens compartidos entre chunks
    z_half = z // 2                                 # Mitad del solapamiento para avanzar la ventana

    # 3) Generación de chunks con solapamiento
    start = 0
    while start < total_tokens:
        end = min(start + chunk_size, total_tokens) # Límite superior del chunk
        chunks.append(tokens[start:end])            # Añadir chunk a la lista
        start += chunk_size - z_half                # Avanzar la ventana teniendo en cuenta el solapamiento

    # 4) Retorno de la lista de chunks
    return chunks

In [16]:
def generate_chunks_legal_structures(tokens, overlap_percent=0.05, min_tokens=512, max_tokens=4096):
    """
    Genera chunks de texto a partir de estructuras legales con solapamiento dinámico.
    Conserva la lógica original de overlapping (z, z_half), evitando exceder el límite de tokens.

    Parámetros:
        tokens (list[str])  : Lista de tokens del texto procesado.
        overlap_percent (float) : Porcentaje de solapamiento dinámico entre chunks.
        min_tokens (int)        : Tamaño mínimo permitido por chunk.
        max_tokens (int)        : Tamaño máximo permitido por chunk.

    Retorna:
        list[list[str]] : Lista de chunks (listas de tokens).
    """
    import re

    # 1) Definición de patrones de división por estructuras legales.
    # Estos patrones permiten identificar divisiones naturales dentro de textos normativos o jurídicos.
    split_patterns = [
        r'LIBRO\s+[IVXLC\d]+',           # División jerárquica mayor (ej: LIBRO PRIMERO)
        r'TÍTULO\s+[IVXLC\d]+',          # División jerárquica principal (ej: TÍTULO II)
        r'CAPÍTULO\s+[IVXLC\d]+',        # División jerárquica media (ej: CAPÍTULO V)
        r'SECCIÓN\s+[IVXLC\d]+',         # Subdivisión de Capítulo o Título
        r'SUBSECCIÓN\s+[IVXLC\d]+',      # Subdivisión de Sección
        r'APARTADO\s+[A-Z\d]+',          # División interna de Artículos o Capítulos
        r'FRACCIÓN\s+[IVXLC\d]+',        # Elemento clave de subdivisión legal
        r'INCISO\s+[A-Z]',               # Elemento de subdivisión menor
        r'Artículo\s+\d+[A-Za-z]*',      # Artículo completo (ej: Artículo 12 Bis)
        r'Art\.\s*\d+[A-Za-z]*',         # Artículo abreviado (ej: Art. 12 Ter)
        r'^\d+\.\s*',                    # Párrafos que inician con numeración (ej: 1. El Tribunal...)
        r'Justificación:?\s*',           # Cabecera común en sentencias y dictámenes
        r'Criterio jurídico:?\s*',       # Cabecera de Tesis o Criterio
        r'Hechos:?\s*',                  # Cabecera de la sección de Antecedentes o Hechos
        r'Fundamento[s]?:?\s*'           # Cabecera de la sección de Sustento Legal
    ]
    combined_pattern = r'(?im)(' + '|'.join(split_patterns) + ')'

    # 2) División inicial del texto en partes basadas en las estructuras detectadas.
    text = ' '.join(tokens)
    splits = re.split(combined_pattern, text)
    splits = [s.strip() for s in splits if s.strip()]

    base_chunks = []
    current_chunk = []

    # 3) Agrupación de tokens según las divisiones encontradas.
    for part in splits:
        if re.match(combined_pattern, part):  # Si es una cabecera legal, inicia un nuevo chunk
            if current_chunk:
                base_chunks.append(current_chunk.copy())
                current_chunk = []
            current_chunk.extend(part.split())
        else:  # Si es contenido, se añade al chunk actual
            current_chunk.extend(part.split())

    if current_chunk:
        base_chunks.append(current_chunk.copy())

    # 4) Subdivisión con solapamiento dinámico para mantener coherencia entre chunks.
    final_chunks = []

    for chunk in base_chunks:
        chunk_size = len(chunk)
        if chunk_size == 0:
            continue

        start = 0
        while start < chunk_size:
            end = min(start + max_tokens, chunk_size) # Límite superior del sub-chunk (sin exceder el máximo permitido)
            sub_chunk = chunk[start:end]              # Extrae la ventana de tokens desde 'start' hasta 'end'

            # Si el sub-chunk resultante es demasiado pequeño, se intenta unir con el anterior.
            if len(sub_chunk) < min_tokens and final_chunks:
                if len(final_chunks[-1]) + len(sub_chunk) <= max_tokens:  # Solo se une si no rebasa el máximo
                    final_chunks[-1].extend(sub_chunk)                    # Se fusiona con el último chunk anterior
                else:
                    final_chunks.append(sub_chunk)                        # Si no cabe, se agrega como nuevo chunk independiente
            else:
                final_chunks.append(sub_chunk)                            # Caso normal: se agrega el sub-chunk directamente

            # 5) Cálculo del solapamiento dinámico.
            z = int(len(sub_chunk) * overlap_percent) # Calcula el número de tokens que deben solaparse
            z_half = z // 2                           # Usa la mitad del solapamiento para suavizar la transición
            start += len(sub_chunk) - z_half          # Avanza el índice con solapamiento parcial (no total)

            # Protección: evita bucles infinitos o posiciones fuera del rango del chunk
            if start >= chunk_size or len(sub_chunk) == 0:
                break  # Sale del bucle si se llegó al final o no hay tokens válidos

    return final_chunks

In [17]:
def process_text_and_generate_chunks(files_df, are_legal_structures=False):
    """
    Procesa archivos para extraer y segmentar el texto en chunks listos para vectorización.

    Parámetros:
        files_df (pd.DataFrame)     : DataFrame con al menos las columnas 'path' y 'name'.
        are_legal_structures (bool) : Si True, usa chunking basado en estructura legal.

    Retorno:
        pd.DataFrame : DataFrame con los chunks generados. Cada fila contiene el nombre del documento,
                       un identificador de chunk, el texto correspondiente y metadata de tokens y chunks.
    """

    # 1) Inicialización de lista donde se almacenarán los chunks generados
    chunks_list = []

    # 2) Iteración sobre cada archivo en el DataFrame para procesar su texto
    for idx, row in tqdm(files_df.iterrows(), total=len(files_df)): # Se usa tqdm para mostrar el progreso del bucle
        path = row['path']
        doc_name = row['name']
        text = extract_text(path)                                   # Extrae el texto del archivo
        clean_tokens = generate_tokens(text)

        # 3) Procesamiento del texto según el tipo de estructura (legal o de expediente)
        if are_legal_structures:                                    # Si se indica que el documento tiene estructura legal
            chunks = generate_chunks_legal_structures(clean_tokens)
        else:                                                       # Si no es un documento legal, aplica un procesamiento estándar
            chunks = generate_chunks(clean_tokens, chunk_size=512)

        # 4) Construcción de la lista de chunks con su respectiva metadata
        for i, chunk in enumerate(chunks):
            chunks_list.append({
                'doc_name': doc_name,
                'text': chunk,
                'chunk_size': len(chunk),
            })

    # 5) Retorno del resultado como DataFrame para facilitar su análisis y vectorización
    return pd.DataFrame(chunks_list)

In [18]:
# Procesamiento de metadatos de archivos de carpetas y vista previa

# 1) Generación de DataFrame chunks
carpetas_chunks_df = process_text_and_generate_chunks(carpetas_df, False)

# 2) Mostrar registros del DataFrame resultante
carpetas_chunks_df_to_display = carpetas_chunks_df.drop(columns=['text'])
carpetas_chunks_df_to_display

100%|██████████| 53/53 [00:38<00:00,  1.37it/s]


,doc_name,chunk_size
0,oficio_a_la_contraloria_del_estado,462
1,correos_electronicos_asegurados,512
2,correos_electronicos_asegurados,512
3,correos_electronicos_asegurados,150
4,oficio_a_la_secretaria_de_finanzas_y_administr...,512
...,...,...
175,acuerdo_de_designacion_de_investigador_a_cargo,445
176,oficio_de_instruccion_del_ministerio_publico,512
177,oficio_de_instruccion_del_ministerio_publico,512
178,oficio_de_instruccion_del_ministerio_publico,512


In [19]:
# Información general del DataFrame
carpetas_chunks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   doc_name    180 non-null    object
 1   text        180 non-null    object
 2   chunk_size  180 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.3+ KB


In [20]:
# Procesamiento de metadatos de archivos de marco jurídico y vista previa

# 1) Generación de DataFrame chunks
marco_juridico_chunks_df = process_text_and_generate_chunks(marco_juridico_df, True)

# 2) Mostrar registros del DataFrame resultante
marco_juridico_chunks_df_to_display = marco_juridico_chunks_df.drop(columns=['text'])
marco_juridico_chunks_df_to_display

100%|██████████| 24/24 [06:00<00:00, 15.03s/it]


,doc_name,chunk_size
0,ley_organica_de_la_fiscalia_general_del_estado...,977
1,ley_organica_de_la_fiscalia_general_del_estado...,4084
2,ley_organica_de_la_fiscalia_general_del_estado...,236
3,ley_organica_de_la_fiscalia_general_del_estado...,709
4,ley_organica_de_la_fiscalia_general_del_estado...,4084
...,...,...
560,jurisprudencia_corrupcion,3974
561,jurisprudencia_corrupcion,4084
562,jurisprudencia_corrupcion,4034
563,jurisprudencia_corrupcion,145


In [21]:
# Información general del DataFrame
marco_juridico_chunks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   doc_name    565 non-null    object
 1   text        565 non-null    object
 2   chunk_size  565 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.4+ KB


# 4. Ingeniería de Características (*Feature Engineering*)

La ingeniería de características es el proceso de dar forma a los datos en un formato que un modelo de aprendizaje automático pueda comprenderlos, y consta específicamente de tres procesos: creación de características, selección de características y transformación de características (Maki & Kaneko, 2025).

Así pues, La ingeniería de características es aún más importante para los datos textuales no estructurados porque se necesita convertir texto de flujo libre en algunas representaciones numéricas, que luego pueden ser entendidas por un algoritmo de aprendizaje automático.

Más allá del proceso estándar que se sigue para procesar datos tabulados, en el caso de datos textuales la ingeniería de características implica el uso de métodos con el objetivo de preservar la semántica, estructura, secuencia y contexto en torno a las palabras cercanas en cada documento (Sarkar, 2019).

En consecuencia, se han propuesto diversos métodos para transformar los datos textuales en representaciones que los modelos puedan procesar. Por ejemplo, existen los métodos llamados tradicionales como *Bag-of-words* o *TF-IDF*. Sin embargo, estos últimos no han demostrado precisión para conservar esa estructura y significado de los documentos.

Recientemente, se utilizan modelos de aprendizaje profundo o *Deep Learning*, lo cuales pueden procesar datos en forma de *Vector Word Embeddings*, que son representaciones numéricas (Gani & Chalaguine, 2022). A estos modelos se les conoce como *Vector Space Models*, y su principal tarea es la de embeber vectores de palabras en un espacio vectorial continuo basándose en la similitud semántica y contextual (Sarkar, 2019).

De esta manera, el reto de este tipo de modelos a la hora de generar los vectores de las palabras, es el poder representar en ciertas dimensiones ya definidas, un Corpus del texto, sin perder el contexto y la similitud semántica entre palabras (Orhan & Tulu, 2021). Estos modelos generan las representaciones vectoriales basados en ventanas de contexto que normalmente es 512 posiciones o *tokens*. Estas ventanas determinan en gran medida el rendimiento y aplicación de los modelos (Nussbaum et al., 2024).

En el presente trabajo como se desarrolló en la sección del *preprocesamiento del corpus de texto*, se obtuvieron *chunks* de diferentes dimensiones en su ventana de contexto, es decir, de diferentes tamaños de caracteres. Esto último, como consecuencia de la lógica de su procesamiento, pues a los textos de **Carpetas de Investigación** al ser textos menos rígidos y un poco más homogéneos en su volumen, se procedió a generar *chunks* de un tamaño estándar tomando en cuenta un proceso de *overlaping* espacial para conservar contexto entre *chunks* adyacentes. Por su parte, los textos del **Marco Jurídico** al ser documentos más rígidos en su estructura, se procesaron mediante un *Chunking* basado en estructuras (artículos, capítulos, libros, etc.), lo cual al final produjo *chunks* que representaron estructuras completas, conservando el contexto, pero haciendo de estos segmentos muy variables en tamaño, aunque se hayan determinado límites mínimos y máximos de *tokens* con el mismo proceso de *overlapping* de los documentos de **Carpetas de Investigación**. En general para ambos tipos de documentos se utilizaron ventanas grandes con el objetivo de preservar el mayor contexto posible.

Para tener eficiencia en el uso de recursos, una uniformidad en el procesamiento de documentos sin importar su naturaleza y un mismo proceso de vectorización de *chunks* que pueden llegar a 4,096 *tokens*, se optó por el uso de un mismo modelo de *embeddings*.

El modelo que se seleccionó fue `e5-base-4k`, propuesto en el artículo llamado "*LongEmbed: Extending Embedding Models for Long Context Retrieval*". Este modelo está basado en *Absolute Position Embedding* (*APE*) que se erige como la estrategia de codificación posicional predominante para los modelos de *embeddings*. Los modelos basados ​​en *APE* primero embeben identificadores de posición absolutos en vectores de posición y añaden *embeddings* de tokens a sus vectores de posición correspondientes, antes de alimentarlos a una pila de capas de transformadores. En el artículo antes mencionado, los autores toman un modelo *APE* base de 512 posiciones y expanden la matriz de *embeddings* de posición para acomodar una mayor capacidad de tokens, y los vectores de *embeddings* están entrenados para representar un rango más amplio de identificadores de posición. `e5-base-4k` puede manejar entradas de hasta 4,096 *tokens*, lo que lo hace perfecto para tareas que requieren una comprensión más profunda del contexto (Zhu et al., 2024).

La elección de este modelo es beneficioso para este proyecto por que permite trabajar con ventanas más grandes a las normales, representa el estado del arte en en modelos de *embeddings* ajustados específicamente para tareas de Recuperación de Información (*Retrieval*), asegurando que los vectores generados para los chunks del **Marco Jurídico** y las **Carpetas de Investigación** sean de alta calidad y se encuentren en un espacio vectorial unificado y coherente.

## 4.1. Normalización

El proceso de **normalización** tuvo como objetivo garantizar que los fragmentos de texto (*chunks*) generados durante la etapa previa de procesamiento tuvieran un formato homogéneo y compatible con el modelo de *embeddings* `e5-base-4k`. Para ello, se implementaron funciones que transforman las listas de *tokens* de cada documento en secuencias textuales unificadas, agregando el prefijo requerido por el modelo (en este caso `"passage: "`), con el fin de orientar la interpretación semántica del texto hacia tareas de recuperación de información.

Esta etapa también permitió realizar una verificación estructural de los *chunks* generados, asegurando que cumplieran con las restricciones de tamaño y formato necesarias para el modelo. Finalmente, se presentaron resúmenes estadísticos y ejemplos de los primeros *chunks* procesados tanto para las **Carpetas de Investigación** como para el **Marco Jurídico**, lo cual permitió evaluar visual y cuantitativamente la calidad del preprocesamiento antes de generar los *embeddings*.

In [22]:
def normalize_chunks_for_e5(df, tokens_column='text'):
    """
    Convierte un DataFrame de chunks en una lista de strings
    preparada para el modelo E5-Base-4k.

    Parámetros:
        df (pd.DataFrame)    : DataFrame que contiene los chunks.
        tokens_column (str)  : Nombre de la columna que contiene la lista de tokens.

    Retorno:
        list : Lista de strings, cada uno con el prefijo "passage: "
    """

    # 1) Validar que la columna existe
    if tokens_column not in df.columns:
        raise ValueError(f"La columna '{tokens_column}' no existe en el DataFrame")

    # 2) Concatenar tokens de cada fila en un string
    chunks_of_text = [' '.join(tokens) for tokens in df[tokens_column]]

    # 3) Agregar el prefijo requerido por E5
    texts_for_e5 = [f"passage: {text}" for text in chunks_of_text]

    return texts_for_e5

In [23]:
def display_chunk_summary(chunks_list, title):
    """
    Muestra un resumen visual de los primeros chunks de texto preparados para E5
    y estadísticas generales de la lista de chunks.

    Parámetros:
        chunks_list (list) : Lista de strings de chunks preparados para E5.
        title (str)        : Título o nombre del conjunto de documentos que se está mostrando.

    Retorno:
        None
    """

    # 1) Mostrar los primeros 3 chunks con un encabezado
    print(f"\n=== Los Primeros 3 Chunks Preparados para E5 de {title} ===\n")
    for i, chunk_text in enumerate(chunks_list[:3]):  # Itera sobre los primeros 3 chunks
        print(f"--- Chunk {i+1} (Longitud: {len(chunk_text)}) ---")
        print("-" * 40)

    # 2) Mostrar resumen general de la lista de chunks
    print(f"\n=== Resumen de la Lista de Chunks de {title} ===")
    print(f"Total de Chunks listos para E5: {len(chunks_list)}")

    # 3) Calcular estadísticas de longitud de los chunks usando NumPy
    chunk_lengths = np.array([len(text) for text in chunks_list]) # Longitud de cada chunk en caracteres
    print(f"Longitud promedio (caracteres) del Chunk: {chunk_lengths.mean():.2f}")
    print(f"Longitud máxima (caracteres) del Chunk: {chunk_lengths.max()}")

In [24]:
# Bloque de normalización y visualización de chunks para E5

# 1) Normalización de los chunks para que sean compatibles con el modelo E5
carpetas_chunk_texts = normalize_chunks_for_e5(carpetas_chunks_df)

# 2) Mostrar un resumen de los primeros chunks y estadísticas generales
display_chunk_summary(carpetas_chunk_texts, "Carpetas de Investigación")


=== Los Primeros 3 Chunks Preparados para E5 de Carpetas de Investigación ===

--- Chunk 1 (Longitud: 2765) ---
----------------------------------------
--- Chunk 2 (Longitud: 2999) ---
----------------------------------------
--- Chunk 3 (Longitud: 2887) ---
----------------------------------------

=== Resumen de la Lista de Chunks de Carpetas de Investigación ===
Total de Chunks listos para E5: 180
Longitud promedio (caracteres) del Chunk: 2602.71
Longitud máxima (caracteres) del Chunk: 3188


In [25]:
# Bloque de normalización y visualización de chunks para E5

# 1) Normalización de los chunks para que sean compatibles con el modelo E5
marco_juridico_chunk_texts = normalize_chunks_for_e5(marco_juridico_chunks_df)

# 2) Mostrar un resumen de los primeros chunks y estadísticas generales
display_chunk_summary(marco_juridico_chunk_texts, "Marco Jurídico")


=== Los Primeros 3 Chunks Preparados para E5 de Marco Jurídico ===

--- Chunk 1 (Longitud: 5757) ---
----------------------------------------
--- Chunk 2 (Longitud: 24763) ---
----------------------------------------
--- Chunk 3 (Longitud: 1401) ---
----------------------------------------

=== Resumen de la Lista de Chunks de Marco Jurídico ===
Total de Chunks listos para E5: 565
Longitud promedio (caracteres) del Chunk: 11917.19
Longitud máxima (caracteres) del Chunk: 24878


## 4.2. Selección

En esta etapa se procedió a la **selección y configuración del modelo de generación de *embeddings***, pieza clave dentro del proceso de *Feature Engineering* al determinar la calidad y profundidad semántica de las representaciones vectoriales obtenidas. Para este proyecto se eligió el modelo **`e5-base-4k`**, disponible públicamente en *Hugging Face* bajo la licencia MIT, debido a su capacidad para procesar secuencias extensas de hasta **4,096 *tokens***, superando el límite convencional de 512 posiciones de modelos *base* y ajustadato para trabajar bajo esquemas de tareas de recuperación.

El modelo `e5-base-4k` fue cargado junto con su *tokenizer* preentrenado, utilizando GPU cuando estaba disponible para optimizar el rendimiento en la etapa de inferencia. Posteriormente, se implementaron funciones auxiliares para manejar adecuadamente los **IDs de posición** (ajustados a las longitudes extendidas del modelo) y aplicar un esquema de **average pooling** sobre los *hidden states*, obteniendo una representación vectorial promedio para cada fragmento de texto (*chunk*).

Finalmente, mediante un proceso por lotes (*batch processing*) se generaron los *embeddings* normalizados para todos los documentos de las **Carpetas de Investigación** y del **Marco Jurídico**, almacenándolos en estructuras tabulares (*DataFrames*) que servirán como base para la etapa de integración con el sistema de recuperación aumentada de información (*RAG*).

In [26]:
# Bloque de inicialización del modelo E5-Base-4k

# 1) Definición del nombre del modelo y del dispositivo de cómputo
MODEL_NAME = 'dwzhu/e5-base-4k'                           # Nombre del modelo a cargar desde Hugging Face
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'   # Selecciona GPU si está disponible, sino CPU

# 2) Carga del tokenizador y del modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)     # Tokenizador preentrenado para E5
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)  # Modelo cargado en el dispositivo seleccionado

# 3) Configuración del modelo en modo evaluación para desactivar dropout y optimizaciones de entrenamiento
model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/225M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [27]:
def average_pool(last_hidden_states, attention_mask):
    """
    Aplica un promedio ponderado (average pooling) sobre los estados ocultos
    de un modelo E5, teniendo en cuenta la máscara de atención para ignorar
    los tokens de padding.

    Parámetros:
        last_hidden_states (Tensor) : Tensor de forma [batch_size, seq_len, hidden_size]
                                      que contiene los embeddings de cada token.
        attention_mask (Tensor)     : Tensor de forma [batch_size, seq_len] donde los
                                      valores 1 indican tokens válidos y 0 tokens de padding.

    Retorno:
        Tensor : Embeddings promedio por secuencia de forma [batch_size, hidden_size].
    """

    # 1) Aplicar máscara para cero en posiciones de padding
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)  # Ceros en tokens no válidos

    # 2) Sumar los embeddings y dividir por el número de tokens válidos para obtener el promedio
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]                  # Pooling promedio

In [28]:
def get_position_ids(input_ids, max_original_positions=512, encode_max_length=4096):
    """
    Genera los IDs de posición para los tokens de entrada, ajustando la escala
    si la secuencia excede el límite original de posiciones del modelo.

    Parámetros:
        input_ids (Tensor)           : Tensor de tokens de forma [batch_size, seq_len].
        max_original_positions (int) : Número máximo de posiciones originales del modelo (default=512).
        encode_max_length (int)      : Longitud máxima de codificación que se desea soportar (default=4096).

    Retorno:
        Tensor : Tensor de IDs de posición expandido a la misma forma que input_ids.
    """

    # 1) Generar IDs de posición secuenciales
    position_ids = list(range(input_ids.size(1)))                 # Lista de 0 a seq_len-1

    # 2) Determinar factor de escalamiento si la secuencia es más larga que el límite original
    factor = max(encode_max_length // max_original_positions, 1)  # Factor mínimo de 1
    if input_ids.size(1) <= max_original_positions:               # Solo escalar si no supera límite original
        position_ids = [(pid * factor) for pid in position_ids]   # Multiplica cada ID por el factor

    # 3) Convertir a tensor y expandir para que coincida con la forma de input_ids
    position_ids = torch.tensor(position_ids, dtype=torch.long)   # Convertir lista a tensor
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids) # Expandir a [batch_size, seq_len]

    # 4) Retornar los IDs de posición listos para usar en el modelo
    return position_ids

In [29]:
def generate_embeddings(chunk_texts, batch_size = 32):
    """
    Genera embeddings para una lista de textos (chunks) usando el modelo E5-Base-4k,
    aplicando tokenización, pooling y normalización.

    Parámetros:
        chunk_texts (list) : Lista de strings, cada uno representando un chunk de texto.
        batch_size (int)   : Número de chunks a procesar en cada lote para eficiencia.

    Retorno:
        pd.DataFrame : DataFrame donde cada fila es el embedding del chunk correspondiente.
    """

    # 1) Inicialización de la lista donde se almacenarán los embeddings
    embeddings = []

    # 2) Procesamiento por batches para evitar saturar la memoria
    for i in tqdm(range(0, len(chunk_texts), batch_size), desc="Vectorizando Chunks"):
        batch_texts = chunk_texts[i:i + batch_size] # Selección del batch actual

        # 2b) Tokenización del batch con truncamiento y padding
        batch_dict = tokenizer(
            batch_texts,
            max_length=4096,  # Limite máximo de tokens del modelo
            padding=True,
            truncation=True,
            return_tensors='pt'
        )

        # 2c) Generar IDs de posición ajustados al tamaño máximo soportado
        batch_dict['position_ids'] = get_position_ids(batch_dict['input_ids'], max_original_positions=512, encode_max_length=4096)

        # 2d) Mover todos los tensores a GPU o CPU según disponibilidad
        batch_dict = {k: v.to(DEVICE) for k, v in batch_dict.items()}

        # 2e) Generación de los hidden states sin gradientes para eficiencia
        with torch.no_grad():
            outputs = model(**batch_dict)

        # 2f) Aplicar average pooling sobre los hidden states
        batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        # 2g) Normalización de los embeddings (necesario para E5)
        batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)

        # 2h) Guardar resultados, convirtiendo a CPU y luego a numpy
        embeddings.extend(batch_embeddings.cpu().numpy())

    # 3) Conversión de la lista de embeddings a array y luego a DataFrame
    embeddings_array = np.array(embeddings)
    embeddings_df = pd.DataFrame(embeddings_array)

    # 4) Retorno del DataFrame con los embeddings
    return embeddings_df

In [30]:
# Bloque de generación y visualización de embeddings

# 1) Generación de embeddings para los chunks de carpetas con batch size de 64
carpetas_embeddings_df = generate_embeddings(carpetas_chunk_texts, batch_size=64)

# 2) Mostrar las primeras filas del DataFrame de embeddings para inspección rápida
carpetas_embeddings_df.head()

Vectorizando Chunks:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/224M [00:00<?, ?B/s]

Vectorizando Chunks: 100%|██████████| 3/3 [00:10<00:00,  3.62s/it]


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.001146,-0.023862,-0.034574,0.011335,0.033912,-0.034609,0.033697,0.018776,-0.022267,-0.034650,...,0.069341,-0.004589,0.043950,-0.065896,0.049208,0.014544,-0.049460,-0.030254,0.030566,0.060313
1,-0.003801,-0.024030,-0.049683,0.007237,0.039069,-0.038349,0.032318,0.037212,-0.025122,-0.036251,...,0.044088,-0.005255,0.059546,-0.064428,0.065043,0.003811,-0.062209,-0.020418,0.052166,0.059677
2,-0.008796,-0.051829,-0.053766,0.003777,0.057954,-0.038604,0.016039,0.044079,-0.023635,-0.017441,...,0.059309,-0.016775,0.054839,-0.063840,0.054772,0.003739,-0.067002,-0.015750,0.024871,0.054183
3,0.004982,-0.038369,-0.039867,0.022432,0.051564,-0.040239,-0.002551,0.052120,-0.016838,-0.005579,...,0.052085,-0.032536,0.037494,-0.041369,0.054898,0.015708,-0.071003,-0.029762,0.025840,0.015169
4,0.001373,-0.015978,-0.035935,0.008555,0.054625,-0.043205,0.027959,0.016223,-0.039832,-0.030940,...,0.070697,-0.005761,0.050527,-0.050591,0.049040,0.015209,-0.044025,-0.027139,0.020114,0.058365


In [31]:
# Información general del DataFrame
carpetas_embeddings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Columns: 768 entries, 0 to 767
dtypes: float32(768)
memory usage: 540.1 KB


In [32]:
# Bloque de generación y visualización de embeddings

# 1) Generación de embeddings para los chunks de carpetas con batch size de 16
marco_juridico_embeddings_df = generate_embeddings(marco_juridico_chunk_texts, batch_size=16)

# 2) Mostrar las primeras filas del DataFrame de embeddings para inspección rápida
marco_juridico_embeddings_df.head()

Vectorizando Chunks: 100%|██████████| 36/36 [04:21<00:00,  7.26s/it]


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.004850,-0.025374,-0.033630,0.016620,0.052890,-0.044678,0.041147,0.033166,-0.039446,-0.009498,...,0.066929,-0.004087,0.054437,-0.070596,0.045841,-0.003528,-0.055047,-0.017354,0.007957,0.058946
1,0.003467,-0.037365,-0.043906,0.004918,0.044713,-0.032258,0.030431,0.016104,-0.038255,-0.030896,...,0.039762,-0.013508,0.050271,-0.067089,0.043559,0.005395,-0.057853,-0.015449,0.005345,0.057845
2,0.009864,-0.038186,-0.005499,-0.007701,0.035786,-0.024040,0.058543,0.024807,-0.031859,-0.034434,...,0.067077,-0.023995,0.042345,-0.058016,0.045036,-0.008908,-0.062560,-0.022936,-0.004833,0.029182
3,-0.012547,-0.042287,-0.042760,0.002941,0.048462,-0.034679,0.036223,0.018511,-0.039407,-0.018870,...,0.055187,-0.015271,0.023413,-0.047577,0.055986,-0.000397,-0.049181,-0.023549,0.006715,0.048287
4,-0.000549,-0.033306,-0.038871,0.009448,0.053195,-0.037230,0.038654,0.007600,-0.052582,-0.030810,...,0.050969,-0.002946,0.043376,-0.056927,0.047304,0.011755,-0.067635,-0.017917,0.017229,0.063662


In [33]:
# Información general del DataFrame
marco_juridico_embeddings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Columns: 768 entries, 0 to 767
dtypes: float32(768)
memory usage: 1.7 MB


# 5. Conclusiones

En este trabajo se procesaron textos de diferentes tipos de documentos provenientes del ámbito legal, con el objetivo de producir representaciones numéricas y generar un espacio vectorial producido por un modelo de *Deep Learning*, con el cual después se proveerá a un sistema *RAG* del contexto necesario para entregar a un *LLM* la información suficiente para dar respuestas a solicitudes concretas.

El proceso que se llevó a cabo para transformar los textos de los documentos de **Carpetas de Investigación** y del **Marco Jurídico**, en representaciones numéricas que los modelos puedan entender, fue a través de la puesta en marcha de técnicas de ingeniería de características o *Feature Engineering* aplicadas a datos textuales. Es decir, después de un proceso de limpieza, *Tokenización* y *Chunking*, se procedidó a generar *embeddings* con un modelo de ventana grande como el `e5-base-4k`.

Los *chunks* generados por el procesamiento de los distintos tipos de documentos, fueron de diversos tamaños, alcanzando límites estipulados de 4,096 *tokens* y aplicando un método de *overlapping* basado en la técnica del artículo "*No Argument Left Behind: Overlapping Chunks for Faster Processing of Arbitrarily Long Legal Texts*" (Fama et al., 2024).

Así, el modelo `e5-base-4k` (distribuido con licencia MIT de código abierto) fue escogido por sus características de poder procesar tokens de diversas dimensiones permitiendo ventanas pequeñas, estándar de 512 posiciones y hasta un límite de 4,096 posiciones como describe el artículo "*LongEmbed: Extending Embedding Models for Long Context Retrieval*" (Zhu et al., 2024). Una vez procesados los textos por el modelo, se generaron *embeddings* de 768 posiciones, las cuales son dimensiones estándar para sistemas *RAG* o *LLM*, lo que representa una ventaja al poder tener representaciones estándar que pueden ajustarse a una variedad de modelos de código abierto. A parte, este modelo está orientado explícitamente a tareas de recuperación, lo cual añade otra ventaja.

En conclusión, en este trabajo se terminó con la preparación de los datos descrita por la metodología **CRISP-ML(Q)** (Studer et al., 2021), en donde se llevó a cabo el ciclo completo de comprensión del dominio, preparación, procesamiento y generación de características para modelos de aprendizaje automático orientados a tareas de recuperación aumentada de información (*RAG*). Este proceso permitió establecer un conjunto de datos estructurado, vectorizado y listo para su integración en un sistema de búsqueda semántica o de recuperación contextual, garantizando la coherencia entre los documentos legales procesados y los futuros modelos de lenguaje que harán uso de dichos embeddings para mejorar la precisión y relevancia de sus respuestas.

# 6. Referencias

- Fama, I., Bueno, B., Alcoforado, A., Ferraz, T. P., Moya, A., & Costa, A. H. R. (2024). *No Argument Left Behind: Overlapping Chunks for Faster Processing of Arbitrarily Long Legal Texts*. arXiv. https://arxiv.org/abs/2410.19184.

- Gani, R., & Chalaguine, L. (2022). *Feature Engineering vs BERT on Twitter Data*. arXiv. https://arxiv.org/abs/2210.16168.

- Maki, J., & Kaneko, H. (2025). Benchmarking automated feature engineering in oxidative coupling of methane and the impact of domain knowledge. *Results in Chemistry*, *18*, 102730. https://doi.org/10.1016/j.rechem.2025.102730.

- Nussbaum, Z, Morris, J., Duderstadt, B., & Mulyar, A. (2024). *Nomic Embed Text: Technical Report*. https://static.nomic.ai/reports/2024_Nomic_Embed_Text_Technical_Report.pdf.

- Orhan, U., & Tulu, C. (2021). A novel embedding approach to learn word vectors by weighting semantic relations: SemSpace. *Expert Systems With Applications*, *180*, 115146. https://doi.org/10.1016/j.eswa.2021.115146.

- Sarkar, D. (2019). *Text Analytics with Python: A Practitioner's Guide to Natural Language Processing*. Apress. https://learning.oreilly.com/library/view/text-analytics-with/9781484243541.

- Studer, S., Bui, T. B., Drescher, C., Hanuschkin, A., Winkler, L., Peters, S., & Mueller, K.-R. (2021). *Towards CRISP-ML(Q): A Machine Learning Process Model with Quality Assurance Methodology*. arXiv. https://doi.org/10.48550/arXiv.2003.05155.

- Zhu, D., Wang, L., Yang, N., Song, Y., Wu, W., Wei, F., & Li, S. (2024). *LongEmbed: Extending Embedding Models for Long Context Retrieval*. arXiv. https://arxiv.org/abs/2404.12096.

- Zhu, D. (2024, 25 de abril). *E5-Base-4k*. Hugging Face. https://huggingface.co/dwzhu/e5-base-4k.